In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy

In [ ]:
print(np.__version__)
print(pd.__version__)
print(sns.__version__)
print(spacy.__version__)

1.26.4
2.1.4
0.13.1
3.7.5


In [ ]:
import sklearn
print(sklearn.__version__)

1.3.2


In [ ]:

!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp=spacy.load('en_core_web_lg')

In [ ]:
!kaggle datasets download prishasawhney/email-classification-ham-spam

Dataset URL: https://www.kaggle.com/datasets/prishasawhney/email-classification-ham-spam
License(s): other
  0% 0.00/4.19k [00:00<?, ?B/s]
100% 4.19k/4.19k [00:00<00:00, 6.79MB/s]


In [ ]:
!unzip email-classification-ham-spam.zip

Archive:  email-classification-ham-spam.zip
  inflating: email_classification.csv  


In [ ]:
df=pd.read_csv("email_classification.csv")

In [ ]:
df.head()

,email,label
0,Upgrade to our premium plan for exclusive acce...,ham
1,Happy holidays from our team! Wishing you joy ...,ham
2,We're hiring! Check out our career opportuniti...,ham
3,Your Amazon account has been locked. Click her...,spam
4,Your opinion matters! Take our survey and help...,ham


In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
le=LabelEncoder()
df.label=le.fit_transform(df.label)

In [ ]:
df.head()

,email,label
0,Upgrade to our premium plan for exclusive acce...,0
1,Happy holidays from our team! Wishing you joy ...,0
2,We're hiring! Check out our career opportuniti...,0
3,Your Amazon account has been locked. Click her...,1
4,Your opinion matters! Take our survey and help...,0


In [ ]:
df.isnull().sum()

email    0
label    0
dtype: int64

In [ ]:
import re
import string


In [ ]:
def to_lowercase(text):
    return text.lower()
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [ ]:
df.email=df.email.apply(to_lowercase)
df.email=df.email.apply(remove_punctuation)

In [ ]:
df['vector']=df['email'].apply(lambda x:nlp(x).vector)


In [ ]:
df.shape

(179, 3)

In [ ]:
df.head()

,email,label,vector
0,upgrade to our premium plan for exclusive acce...,0,"[-1.1136907, 2.0064514, -4.32895, -0.81849706,..."
1,happy holidays from our team wishing you joy a...,0,"[1.3240365, 0.8890292, -3.0343075, -0.8895452,..."
2,were hiring check out our career opportunities...,0,"[-0.034031708, 1.2652917, -2.8660052, 0.878722..."
3,your amazon account has been locked click here...,1,"[1.7271876, 2.4194536, -2.7477725, 0.27438924,..."
4,your opinion matters take our survey and help ...,0,"[1.9549226, 0.6181291, -5.4887137, -0.79988694..."


In [ ]:
df.drop('email',axis=1,inplace=True)

In [ ]:
df.head()

,label,vector
0,0,"[-1.1136907, 2.0064514, -4.32895, -0.81849706,..."
1,0,"[1.3240365, 0.8890292, -3.0343075, -0.8895452,..."
2,0,"[-0.034031708, 1.2652917, -2.8660052, 0.878722..."
3,1,"[1.7271876, 2.4194536, -2.7477725, 0.27438924,..."
4,0,"[1.9549226, 0.6181291, -5.4887137, -0.79988694..."


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.vector,df.label,test_size=0.2,random_state=42)

In [ ]:
x_train.shape

(143,)

In [ ]:
x_train2d=np.array(x_train.to_list())
x_test2d=np.stack(x_test)
x_train2d.shape,x_test2d.shape

((143, 300), (36, 300))

In [ ]:
y_train.shape,y_test.shape

((143,), (36,))

In [ ]:
scaler=MinMaxScaler();
scale_x_train=scaler.fit_transform(x_train2d)
scale_x_test=scaler.fit_transform(x_test2d)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(scale_x_train,y_train)

MultinomialNB()

In [ ]:
df.label.value_counts()

label
0    100
1     79
Name: count, dtype: int64

In [ ]:
df.label.unique()

array([0, 1])

In [ ]:
label=['ham','spam']

In [ ]:
y_pred=model.predict(scale_x_test)

In [ ]:
df.label[:5]

0    0
1    0
2    0
3    1
4    0
Name: label, dtype: int64

In [ ]:
y_pred[:5]

array([1, 1, 1, 1, 1])

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[14  0]
 [ 4 18]]
              precision    recall  f1-score   support

           0       0.78      1.00      0.88        14
           1       1.00      0.82      0.90        22

    accuracy                           0.89        36
   macro avg       0.89      0.91      0.89        36
weighted avg       0.91      0.89      0.89        36



In [ ]:
from xgboost import XGBClassifier
model=XGBClassifier()
model.fit(x_train2d,y_train)
y_pred=model.predict(x_test2d)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[12  2]
 [ 3 19]]
              precision    recall  f1-score   support

           0       0.80      0.86      0.83        14
           1       0.90      0.86      0.88        22

    accuracy                           0.86        36
   macro avg       0.85      0.86      0.86        36
weighted avg       0.86      0.86      0.86        36



In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train2d,y_train)
y_pred=model.predict(x_test2d)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[14  0]
 [ 0 22]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        22

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
model.fit(x_train2d,y_train)
y_pred=model.predict(x_test2d)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[13  1]
 [ 2 20]]
              precision    recall  f1-score   support

           0       0.87      0.93      0.90        14
           1       0.95      0.91      0.93        22

    accuracy                           0.92        36
   macro avg       0.91      0.92      0.91        36
weighted avg       0.92      0.92      0.92        36



In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train2d,y_train)
y_pred=model.predict(x_test2d)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[14  0]
 [ 0 22]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        22

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



In [ ]:
import pickle

# Assuming model is your trained RandomForestClassifier
with open('emailspam.pkl', 'wb') as file:
    pickle.dump(model, file)


In [ ]:
import pickle

# Load the model from the file
with open('emailspam.pkl', 'rb') as file:
    newmodel = pickle.load(file)


In [ ]:
y_pred=newmodel.predict(x_test2d)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[14  0]
 [ 0 22]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        22

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



In [ ]:
df1 = pd.read_csv("spam.csv", encoding='latin1')

In [ ]:
df1.shape

(5572, 5)

In [ ]:
df1.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [ ]:
df1.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df1.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [ ]:
df2=df1.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1)

In [ ]:
df2.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df2.v1=le.fit_transform(df2.v1)
df2.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df2.isnull().sum()

v1    0
v2    0
dtype: int64

In [ ]:
df2.duplicated().sum()

403

In [ ]:
df2.drop_duplicates(inplace=True)

In [ ]:
df2.duplicated().sum()

0

In [ ]:
df2.isnull().sum()

v1    0
v2    0
dtype: int64

In [ ]:
df2.shape

(5169, 2)

In [ ]:
def to_lowercase(text):
    return text.lower()
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [ ]:
df2.v2=df2.v2.apply(to_lowercase)
df2.v2=df2.v2.apply(remove_punctuation)

In [ ]:
df2['vector']=df2['v2'].apply(lambda x:nlp(x).vector)

In [ ]:
df3=df2.drop('v2',axis=1)

In [ ]:
df3.head()

,v1,vector
0,0,"[0.57386875, 1.6531525, -0.27780083, -0.302871..."
1,0,"[-0.33230335, 1.0907167, 0.21502167, -0.99249,..."
2,1,"[-0.6612257, 0.847962, -0.7202453, 0.36291173,..."
3,0,"[-1.7141063, 2.14568, 0.32646608, -1.2899854, ..."
4,0,"[0.6140064, 4.7306733, -3.5350685, -1.5889207,..."


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(df3.vector,df3.v1,test_size=0.2,random_state=42)

In [ ]:
X_train.shape

(4135,)

In [ ]:
X_train2d=np.stack(X_train)
X_test2d=np.stack(X_test)

In [ ]:
X_train2d.shape,X_test2d.shape

((4135, 300), (1034, 300))

In [ ]:
scaler=MinMaxScaler();
scale_X_train=scaler.fit_transform(X_train2d)
scale_X_test=scaler.fit_transform(X_test2d)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(scale_X_train,Y_train)

MultinomialNB()

In [ ]:
Y_pred=model.predict(scale_X_test)

In [ ]:
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))
print(model.score(scale_X_test,Y_test))
print(model.score(scale_X_train,Y_train))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93       889
           1       0.65      0.14      0.23       145

    accuracy                           0.87      1034
   macro avg       0.76      0.56      0.58      1034
weighted avg       0.84      0.87      0.83      1034

[[878  11]
 [125  20]]
0.8684719535783365
0.8769044740024183


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(X_train2d,Y_train)
y_pred=model.predict(X_test2d)
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(model.score(X_test2d,Y_test))
print(model.score(X_train2d,Y_train))

[[878  11]
 [125  20]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       889
           1       0.65      0.14      0.23       145

    accuracy                           0.87      1034
   macro avg       0.76      0.56      0.58      1034
weighted avg       0.84      0.87      0.83      1034

0.960348162475822
0.999758162031439


In [ ]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
model.fit(X_train2d,Y_train)
y_pred=model.predict(X_test2d)
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(model.score(X_test2d,Y_test))
print(model.score(X_train2d,Y_train))

[[878  11]
 [125  20]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       889
           1       0.65      0.14      0.23       145

    accuracy                           0.87      1034
   macro avg       0.76      0.56      0.58      1034
weighted avg       0.84      0.87      0.83      1034

0.9206963249516441
0.999758162031439


In [ ]:
from xgboost import XGBClassifier
xgm=XGBClassifier()
xgm.fit(X_train2d,Y_train)
y_pred=xgm.predict(X_test2d)
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(xgm.score(X_test2d,Y_test))
print(xgm.score(X_train2d,Y_train))

[[878  11]
 [125  20]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       889
           1       0.65      0.14      0.23       145

    accuracy                           0.87      1034
   macro avg       0.76      0.56      0.58      1034
weighted avg       0.84      0.87      0.83      1034

0.9729206963249516
0.999758162031439


In [ ]:
with open('EmailSpam2.pkl', 'wb') as file:
    pickle.dump(xgm, file)

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train2d,Y_train)
y_pred=model.predict(X_test2d)
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(model.score(X_test2d,Y_test))
print(model.score(X_train2d,Y_train))

[[878  11]
 [125  20]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       889
           1       0.65      0.14      0.23       145

    accuracy                           0.87      1034
   macro avg       0.76      0.56      0.58      1034
weighted avg       0.84      0.87      0.83      1034

0.9545454545454546
0.9883917775090689


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=5)
model.fit(X_train2d,Y_train)
y_pred=model.predict(X_test2d)
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(model.score(X_test2d,Y_test))
print(model.score(X_train2d,Y_train))

[[878  11]
 [125  20]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       889
           1       0.65      0.14      0.23       145

    accuracy                           0.87      1034
   macro avg       0.76      0.56      0.58      1034
weighted avg       0.84      0.87      0.83      1034

0.9400386847195358
0.9574365175332528


In [ ]:
from sklearn.svm import SVC
model=SVC(kernel='rbf')
model.fit(X_train2d,Y_train)
y_pred=model.predict(X_test2d)
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(model.score(X_test2d,Y_test))
print(model.score(X_train2d,Y_train))

[[878  11]
 [125  20]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       889
           1       0.65      0.14      0.23       145

    accuracy                           0.87      1034
   macro avg       0.76      0.56      0.58      1034
weighted avg       0.84      0.87      0.83      1034

0.97678916827853
0.9823458282950424
